In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(ngram_range=(1, 5))

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, BernoulliNB  # CategoricalNB not compatible with sparse matrix
from sklearn.model_selection import train_test_split

In [5]:
from sklearn.metrics import f1_score

In [6]:
database = pd.read_csv('./train.csv', low_memory=False)

In [7]:
database.dropna(inplace=True)

In [8]:
database.shape

(224125, 14)

In [9]:
df, dt = train_test_split(database, train_size=0.1)

In [10]:
df.shape

(22412, 14)

In [11]:
vect.fit(df['comment'])

CountVectorizer(ngram_range=(1, 5))

In [12]:
print("Vocabulary size: {}".format(len(vect.vocabulary_)))
#print("Vocabulary content:\n {}".format(vect.vocabulary_))

Vocabulary size: 705141


In [13]:
X_train = vect.transform(df['comment'])
X_train.shape

(22412, 705141)

In [14]:
X_test = vect.transform(dt['comment'])
X_test.shape

(201713, 705141)

In [15]:
df.columns[7:]

Index(['price_value', 'fake_originality', 'warranty', 'size', 'discrepancy',
       'flavor_odor', 'expiration_date'],
      dtype='object')

In [16]:
vec_train = pd.DataFrame()
model_BernoulliNB = {}
model_MultinomialNB = {}
model_LogReg = {}
for col in df.columns[7:]:
    vec_train[col] = df[col].astype('bool')
    model_BernoulliNB[col] = BernoulliNB().fit(X_train, vec_train[col])
    model_MultinomialNB[col] = MultinomialNB().fit(X_train, vec_train[col])
    model_LogReg[col] = LogisticRegression().fit(X_train, vec_train[col])

In [17]:
pred = []
for key, Bmod, Mmod, Lmod in zip(model_BernoulliNB.keys(),
                           model_BernoulliNB.values(),
                           model_MultinomialNB.values(),
                           model_LogReg.values()):
    
    pred.append( (key,
                  f1_score(Bmod.predict(X_train), vec_train[key]).round(3),
                  f1_score(Mmod.predict(X_train), vec_train[key]).round(3),
                  f1_score(Lmod.predict(X_train), vec_train[key]).round(3),) )
print(*pred, sep='\n')

('price_value', 0.342, 0.962, 0.981)
('fake_originality', 0.0, 0.874, 0.956)
('warranty', 0.0, 0.892, 0.974)
('size', 0.136, 0.971, 0.988)
('discrepancy', 0.069, 0.977, 0.995)
('flavor_odor', 0.044, 0.943, 0.988)
('expiration_date', 0.0, 0.887, 0.975)


In [18]:
pred = []
vec_test = {}
for key, Bmod, Mmod, Lmod in zip(model_BernoulliNB.keys(),
                           model_BernoulliNB.values(),
                           model_MultinomialNB.values(),
                           model_LogReg.values()):
    vec_test[key] = dt[key].astype('bool')
    pred.append( (key,
                  f1_score(Bmod.predict(X_test), vec_test[key]).round(3),
                  f1_score(Mmod.predict(X_test), vec_test[key]).round(3),
                  f1_score(Lmod.predict(X_test), vec_test[key]).round(3),) )
print(*pred, sep='\n')

('price_value', 0.006, 0.663, 0.807)
('fake_originality', 0.0, 0.013, 0.19)
('warranty', 0.0, 0.004, 0.145)
('size', 0.001, 0.181, 0.652)
('discrepancy', 0.0, 0.142, 0.565)
('flavor_odor', 0.002, 0.317, 0.79)
('expiration_date', 0.0, 0.027, 0.355)


<div dir="rtl">
    <font size=4>
    نکته اساسی در استفاده از مدل های نایو بیز استقلال متغیرها از یکدیگر است بر این اساس در زمان کار با متن نیاز است که این نکته به دقت بررسی شود. زیرا که به صورت کلی و ذاتی کلمات در زبان انسان از هم مستقل نیستند و در ارتباط با یکدیگر معنا می سازند و بخصوص این نکته در فعل های مثبت و منفی خود را نشان می دهند. در نتیجه هرچی موضوع ما به ماهیت معنایی جملات ارتباط بیشتری داشته باشد اعتبار مدل های نیو بیز کمتر می شود. در بررسی بالا هم مشاهده می شود که هر چه موضوع کلی تر بوده دقت مدل های نایو بیز هم بالاتر رفته اما استفاده از مدل لاجستیک ریگریشن با توجه به بررسی اثر تقابل متغیرها در تمامی حالات بهتر عمل کرده اما هرچه اثر معنایی جملات در برچسب ها بالاتر رفته، این مدل نیز که به دنبال یک رابطه خطی هست ضعیف تر عملکرده و در واقع نیاز به بررسی های نحوی متن از یک سو بیشتر مورد لزوم به نظر می رسد و از سوی دیگر نیاز به مدل های غیر خطی برای تفکیک بهتر ارتباط معنایی کلمات با هم  
    </font>
</div>

<div dir="rtl">
    <font size=4>
        جدای از این مسئله مفهومی، اعداد بالا که سری اول دقت مدل ها روی داده های تعلیم و سری دوم دقت روی داده های تست با این نکته که بخاطر حجم بالا در این جا 10 درصد داده ها برای تعلیم استفاده شده است. نشان می دهد که مدل نایو بیز در اینجا یا اصلن دارای underfitting هست و یا که شدیدن دچار overfitting زیرا که مستقل از دقت مدل در داده های تعلیم، دقت افت شدیدی در داده های تست متحمل شده است.  
    </font>
</div>

In [25]:
from scipy import sparse
from sklearn.pipeline import make_pipeline

<div dir="rtl">
    <font size=4>
        با اضافه کردن دسته بندی محصولات به کامنت ها و استفاده از درخت های تصمیم گیری به خاطر سبک تر شدن محاسبات نسبت به لاجستیک ریگریشن عملکرد بهتری در ادامه کسب شده است.  
    </font>
</div>

In [20]:
database.category_id = database.category_id.astype('category').cat.codes
database.product_id = database.product_id.astype('category').cat.codes
database.is_buyer = database.is_buyer == 'True'
for col in database.columns[7:]:
    database[col] = database[col].astype('bool')

In [21]:
train, test = train_test_split(database)

In [41]:
models = {}
for ngram_ranges in [(1, 1), (1, 2), (1, 3), (1, 4), (1, 5)]:
    model_scores = [('col', 'train score', 'test score')]
    print(ngram_ranges)
    print(*(f'\t{txt:17}' for txt in model_scores[-1]))
    vectorizer = CountVectorizer(ngram_range=ngram_ranges)
    X = sparse.hstack([vectorizer.fit_transform(train.comment), train[['category_id']].astype('category')])
    X_t = sparse.hstack([vectorizer.transform(test.comment), test[['category_id']].astype('category')])
    for col in df.columns[7:]:
        model_M = MultinomialNB().fit(X, train[col])
        model_scores.append((col,
                            f1_score(train[col], model_M.predict(X)).round(3),
                            f1_score(test[col], model_M.predict(X_t)).round(3),
                            ))
        print(*(f'\t{str(txt):17}' for txt in model_scores[-1]))
    models[ngram_ranges] = model_scores

(1, 1)
	col               	train score       	test score       
	price_value       	0.691             	0.661            
	fake_originality  	0.06              	0.071            
	warranty          	0.052             	0.057            
	size              	0.469             	0.409            
	discrepancy       	0.315             	0.307            
	flavor_odor       	0.7               	0.649            
	expiration_date   	0.018             	0.024            
(1, 2)
	col               	train score       	test score       
	price_value       	0.592             	0.508            
	fake_originality  	0.048             	0.03             
	warranty          	0.053             	0.012            
	size              	0.303             	0.19             
	discrepancy       	0.231             	0.191            
	flavor_odor       	0.191             	0.063            
	expiration_date   	0.005             	0.017            
(1, 3)
	col               	train score       	test score       
	price_val

<div dir="rtl">
    <font size=4>
        در بالا مشاهده میشه که با افزایش ان-گرام ها در مرحله اول برای ایجاد ترکیبات معادله معانی جملات این کار در روش نایو بیز خوب عمل کرده و باعث افت شدید نتیجه ها شده است.
    </font>
</div>

In [22]:
from sklearn.tree import DecisionTreeClassifier

<div dir="rtl">
    <font size=4>
        برای بررسی ساده و کاهش ابعاد مسئله ابتدا سراغ ستون بو و مزه رفته و با بررسی کامنت های مثبت و ایجاد یک دیکشنری دستی سبک به عملکرد درخت تصمیم گیری پرداخته شده است.
    </font>
</div>

In [43]:
words = set(['خوش', 'خوشمزه', 'خوش مزه', 'خوشمزس', 'بو', 'بوی', 'مزه', 'مزی', 'بوش', 'عطر', 'معطر', 'خوشبو', 'رایحه',
         'طعم', 'طعمش', 'خوشمزه', 'رایحه', 'شیرین', 'تند', 'شور', 'تلخ', 'مزس', 'خوشمزست', 'شیرینه', 'طعمه', 'اسانس', 'بوشم'
         , 'خوشمزههه', 'خوشبوء', 'طعمه', 'تنده'])
subject = 'flavor_odor'
print(subject)
vectorizer = CountVectorizer(binary=True, vocabulary=dict(zip(words,range(len(words)))))
X = sparse.hstack([vectorizer.transform(train.comment), train[['category_id']].astype('category')])
X_t = sparse.hstack([vectorizer.transform(test.comment), test[['category_id']].astype('category')])
price_value_Mul_all = DecisionTreeClassifier().fit(X, train[subject])
print(f'\t#train:{train.shape[0]}  #test:{test.shape[0]}')
print('\t[per recall] train:', f1_score(train[subject], price_value_Mul_all.predict(X), average=None).round(3))
print('\t[per recall] test:', f1_score(test[subject], price_value_Mul_all.predict(X_t), average=None).round(3))
print('\tf1 train:', f1_score(train[subject], price_value_Mul_all.predict(X)).round(3))
print('\tf1 test:', f1_score(test[subject], price_value_Mul_all.predict(X_t)).round(3))

flavor_odor
	#train:168093  #test:56032
	[per recall] train: [0.995 0.909]
	[per recall] test: [0.993 0.87 ]
	f1 train: 0.909
	f1 test: 0.87


<div dir="rtl">
    <font size=4>
        با یک دیکشنری کوچک و محاسبات سبک مشاهده می شود که این مدل با دقت بسیار خوبی در باره این ستون تصمیم گیری کرده است.  اما خوب ساختن دیکشنری به صورت دستی عملن در یادگیری ماشین کار صحیحی نیست و یک ایده برای سبک کردن این دیکشنری و کاهش محاسبات لازم است. 
    </font>
</div>

In [50]:
from itertools import filterfalse
from sklearn.ensemble import RandomForestClassifier

<div dir="rtl">
    <font size=4>
        
    </font>
</div>

In [55]:
ngram_range_set = (1, 2)
vectorizer = CountVectorizer(binary=True, ngram_range=ngram_range_set).fit(train.comment)
X = sparse.hstack([vectorizer.transform(train.comment), train[['category_id']].astype('category')])
X_t = sparse.hstack([vectorizer.transform(test.comment), test[['category_id']].astype('category')])
Mul_all = RandomForestClassifier(n_estimators=9,
                                 max_depth=7,
                                 min_samples_split=0.25,
                                 min_samples_leaf=10,
                                 n_jobs=-1,
                                ).fit(X, train.iloc[:, -7:])
print(f'\t#train:{train.shape[0]}  #test:{test.shape[0]}')
print('\t[per recall] train:', f1_score(train.iloc[:, -7:], Mul_all.predict(X), average=None).round(3))
print('\t[per recall] test:', f1_score(test.iloc[:, -7:], Mul_all.predict(X_t), average=None).round(3))

	#train:168093  #test:56032
	[per recall] train: [0. 0. 0. 0. 0. 0. 0.]
	[per recall] test: [0. 0. 0. 0. 0. 0. 0.]


In [63]:
subject = 'flavor_odor'
print(subject)
ngram_range_set = (2, 7)
price_value_vc_p = CountVectorizer(binary=True, ngram_range=ngram_range_set).fit(train.loc[train[subject]==1.0, 'comment'])
price_value_vc_n = CountVectorizer(binary=True, ngram_range=ngram_range_set).fit(train.loc[train[subject]==0.0, 'comment'])
my_dict = {ngram:valv for valv, ngram in enumerate(
    filterfalse(lambda ngram: ngram in price_value_vc_n.vocabulary_.keys(),
               price_value_vc_p.vocabulary_.keys()))}
vectorizer = CountVectorizer(binary=True, ngram_range=ngram_range_set, vocabulary=my_dict)
X = sparse.hstack([vectorizer.transform(train.comment), train[['category_id']].astype('category')])
X_t = sparse.hstack([vectorizer.transform(test.comment), test[['category_id']].astype('category')])

flavor_odor


In [64]:
Mul_all = RandomForestClassifier(n_estimators=13,
                                 n_jobs=-1,
                                ).fit(X, train[subject])
print(f'\t#train:{train.shape[0]}  #test:{test.shape[0]}')
print('\t[per recall] train:', f1_score(train[subject], Mul_all.predict(X), average=None).round(3))
print('\t[per recall] test:', f1_score(test[subject], Mul_all.predict(X_t), average=None).round(3))
print('\tf1 train:', f1_score(train[subject], Mul_all.predict(X)).round(3))
print('\tf1 test:', f1_score(test[subject], Mul_all.predict(X_t)).round(3))

	#train:168093  #test:56032
	[per recall] train: [0.995 0.907]
	[per recall] test: [0.98  0.409]
	f1 train: 0.907
	f1 test: 0.409


In [65]:
X = sparse.hstack([price_value_vc_p.transform(train.comment), train[['category_id']].astype('category')])
X_t = sparse.hstack([price_value_vc_p.transform(test.comment), test[['category_id']].astype('category')])
Mul_all = RandomForestClassifier(n_estimators=13,
                                 n_jobs=-1,
                                ).fit(X, train[subject])
print(f'\t#train:{train.shape[0]}  #test:{test.shape[0]}')
print('\t[per recall] train:', f1_score(train[subject], Mul_all.predict(X), average=None).round(3))
print('\t[per recall] test:', f1_score(test[subject], Mul_all.predict(X_t), average=None).round(3))
print('\tf1 train:', f1_score(train[subject], Mul_all.predict(X)).round(3))
print('\tf1 test:', f1_score(test[subject], Mul_all.predict(X_t)).round(3))

	#train:168093  #test:56032
	[per recall] train: [0.999 0.978]
	[per recall] test: [0.989 0.749]
	f1 train: 0.978
	f1 test: 0.749


In [66]:
ngram_range_set = (2, 7)
for subject in train.columns[-7:]:
    print(subject)
    vectorizer = CountVectorizer(binary=True, ngram_range=ngram_range_set).fit(train.loc[train[subject]==1.0, 'comment'])
    X = sparse.hstack([vectorizer.transform(train.comment), train[['category_id']].astype('category')])
    X_t = sparse.hstack([vectorizer.transform(test.comment), test[['category_id']].astype('category')])
    Mul_all = RandomForestClassifier(n_estimators=13,
                                 n_jobs=-1,
                                ).fit(X, train[subject])
    print('\t[per recall] train:', f1_score(train[subject], Mul_all.predict(X), average=None).round(3))
    print('\t[per recall] test:', f1_score(test[subject], Mul_all.predict(X_t), average=None).round(3))
    print('\tf1 train:', f1_score(train[subject], Mul_all.predict(X)).round(3))
    print('\tf1 test:', f1_score(test[subject], Mul_all.predict(X_t)).round(3))

price_value
	[per recall] train: [0.996 0.983]
	[per recall] test: [0.946 0.745]
	f1 train: 0.983
	f1 test: 0.745
fake_originality
	[per recall] train: [1.    0.948]
	[per recall] test: [0.997 0.269]
	f1 train: 0.948
	f1 test: 0.269
warranty
	[per recall] train: [1.    0.935]
	[per recall] test: [0.999 0.097]
	f1 train: 0.935
	f1 test: 0.097
size
	[per recall] train: [0.997 0.97 ]
	[per recall] test: [0.972 0.531]
	f1 train: 0.97
	f1 test: 0.531
discrepancy
	[per recall] train: [0.998 0.961]
	[per recall] test: [0.981 0.273]
	f1 train: 0.961
	f1 test: 0.273
flavor_odor
	[per recall] train: [0.999 0.976]
	[per recall] test: [0.988 0.736]
	f1 train: 0.976
	f1 test: 0.736
expiration_date
	[per recall] train: [1.    0.957]
	[per recall] test: [0.999 0.373]
	f1 train: 0.957
	f1 test: 0.373
